In [1]:
import setup

setup.init_django()

In [2]:
from rag import (
    db as rag_db, 
    engines as rag_engines,
    settings as rag_settings, 
    updaters as rag_updaters,
    patches as rag_patches,
)

In [3]:
from typing import Optional, Union
from sqlalchemy import create_engine, text

In [4]:
rag_settings.init()
rag_db.init_vector_db()
rag_updaters.update_llama_index_documents(use_saved_embeddings=True)

In [5]:
vector_index = rag_engines.get_semantic_query_index()
semantic_query_retriever = rag_engines.get_semantic_query_retriever_engine()
sql_query_engine = rag_engines.get_sql_query_engine()

In [6]:
print(rag_settings.VECTOR_DB_NAME, rag_settings.VECTOR_DB_TABLE_NAME)

vector_db blogpost


In [7]:
from llama_index.core.tools import QueryEngineTool

vector_tool = QueryEngineTool.from_defaults(
    query_engine=semantic_query_retriever,
    description=(
        f"Useful for answering semantic questions about different blog posts"
    ),
)

In [8]:
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over"
        " a table containing: blog posts and page views each blog post"
    ),
)

In [9]:
query_engine = rag_patches.MySQLAutoVectorQueryEngine(
    sql_tool, 
    vector_tool,
)

In [10]:
response = query_engine.query(
    "what is americano?"
)

Querying other query engine: The question 'what is americano?' is a semantic question that is more likely to be answered by a system designed to handle semantic queries about different blog posts.


In [11]:
response.response

'I\'m sorry, but I don\'t have any information about "americano" based on the information available to me.'

In [13]:
response = query_engine.query(
    "Who is the bartender?"
)

Querying other query engine: The question 'Who is the bartender?' is a semantic question that is more likely to be answered by a system designed to handle semantic queries about different blog posts.


In [14]:
from IPython.display import Markdown, display

display(Markdown(response.response))

There is no bartender mentioned.